In [71]:
import re
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('averaged_perceptron_tagger')

from string import punctuation

punctuation += '«»—…'

!pip install pymystem3 pymorphy2

from pymystem3 import Mystem

m = Mystem()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
with open('our_corpus.txt', encoding="utf-8") as data:
    text = str(data.read())

docs = text.split('%')
docs = docs[1:]

In [83]:
genders = []
doc_ids = []
doc_names = []
doc_texts = []
sent_ids = []
sentence_texts = []
sent_positions = []
word_ids = []
words = []
lemmas = []
tags = []
word_positions = []

sent_id = 0
word_id = 0
sentences_all = []

for doc in docs:

  text = sent_tokenize(doc)
  doc_name = text[1]
  doc_text = doc
  gender = doc[0]
  sentences = text[2:]
  word_position = 0

  for s in sentences:
    sentence_text = s

    sent_position = sentences.index(s) + 1
    sent_id += 1

    sentences_all.append(s)

    for w in m.analyze(s):
      if 'analysis' in w:
        if w['analysis'] != []:
          word_id += 1
          word_position += 1

          word = w['text']
          lemma = w['analysis'][0]['lex']
          gr = w['analysis'][0]['gr']
          pos = gr.split('=')[0].split(',')[0]

          tags.append(pos)
          words.append(word.lower())
          lemmas.append(lemma)
          doc_names.append(doc_name)
          doc_texts.append(doc_text)
          sentence_texts.append(sentence_text)
          sent_positions.append(sent_position)
          sent_ids.append(sent_id)
          word_positions.append(word_position)
          genders.append(gender)
          doc_ids.append(docs.index(doc) + 1)
          word_ids.append(word_id)

        else:
          if w['text'].isalpha():
            word_id += 1
            word_position += 1

            word = w['text']
            lemma = m.lemmatize(w['text'])[0]
            pos = 'LAT'

            tags.append(pos)
            words.append(word)
            lemmas.append(lemma)
            doc_names.append(doc_name)
            doc_texts.append(doc_text)
            sentence_texts.append(sentence_text)
            sent_positions.append(sent_position)
            word_positions.append(word_position)
            genders.append(gender)
            doc_ids.append(docs.index(doc) + 1)
            sent_ids.append(sent_id)
            word_ids.append(word_id)

In [84]:
print(len(words))

10022


In [95]:
inverted_word_indexes = []

for w in words:
  inverted_index = []
  for s in sentences_all:
    if w.lower() in s.lower():
      inverted_index.append(sentences_all.index(s) + 1)
  inverted_word_indexes.append(inverted_index)

In [86]:
lemmatized_sentences_all = []

for s in sentences_all:
  ana_s = m.analyze(s)
  words_for_s = []
  for w in ana_s:
    if 'analysis' in w:
      if w['analysis'] != []:
        words_for_s.append(w['analysis'][0]['lex'])
      else:
        words_for_s.append(m.lemmatize(w['text'])[0])

  lemmatized_sentences_all.append(' '.join(words_for_s))

In [73]:
len(lemmatized_sentences_all)

525

In [91]:
inverted_lemma_indexes = []

for w in lemmas:
  inverted_index = []
  for s in lemmatized_sentences_all:
    if w.lower() in s.lower():
      inverted_index.append(lemmatized_sentences_all.index(s) + 1)
  inverted_lemma_indexes.append(inverted_index)

In [96]:
corpus_dict = {
    'gender': genders,
    'doc_id': doc_ids,
    'doc_name': doc_names,
    'doc_text': doc_texts,
    'sent_id': sent_ids,
    'sentence_text': sentence_texts,
    'sent_position': sent_positions,
    'word_id': word_ids,
    'word': words,
    'lemma': lemmas,
    'tag': tags,
    'word_position': word_positions,
    'inverted_word_index': inverted_word_indexes,
    'inverted_lemma_index': inverted_lemma_indexes
}

corpus_df = pd.DataFrame.from_dict(corpus_dict)

In [97]:
corpus_df

,gender,doc_id,doc_name,doc_text,sent_id,sentence_text,sent_position,word_id,word,lemma,tag,word_position,inverted_word_index,inverted_lemma_index
0,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,1,из-за,из-за,PR,1,"[1, 24, 60, 440]","[1, 24, 60, 440]"
1,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,2,того,то,SPRO,2,"[1, 4, 8, 53, 60, 73, 121, 134, 144, 161, 221,...","[1, 2, 4, 8, 11, 13, 18, 22, 24, 26, 29, 32, 3..."
2,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,3,что,что,CONJ,3,"[1, 11, 26, 53, 55, 60, 73, 87, 94, 101, 104, ...","[1, 11, 26, 53, 55, 60, 73, 87, 94, 101, 104, ..."
3,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,4,смыслы,смысл,S,4,"[1, 81, 82, 315]","[1, 10, 18, 34, 81, 82, 129, 130, 151, 165, 16..."
4,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,5,не,не,PART,5,"[1, 2, 4, 5, 9, 10, 11, 14, 18, 21, 24, 26, 28...","[1, 2, 4, 5, 9, 10, 11, 14, 18, 21, 24, 26, 28..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10017,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10018,значения,значение,S,205,"[4, 11, 28, 32, 53, 68, 156, 159, 169, 190, 23...","[4, 9, 11, 28, 32, 34, 52, 53, 56, 68, 81, 99,..."
10018,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10019,или,или,CONJ,206,"[10, 11, 26, 34, 40, 41, 50, 56, 60, 71, 76, 1...","[10, 11, 26, 34, 40, 41, 50, 60, 71, 76, 101, ..."
10019,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10020,формы,форма,S,207,[525],"[6, 18, 60, 63, 97, 98, 199, 223, 224, 348, 35..."
10020,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,525,Семантические фраземы:\n1) Идиомы (≈ сращения ...,13,10021,составных,составной,A,208,[525],[525]


In [99]:
corpus_df.to_csv('nlp_project_corpus_df.csv')

In [98]:
LATs = corpus_df[corpus_df.tag == 'LAT']
LATs

,gender,doc_id,doc_name,doc_text,sent_id,sentence_text,sent_position,word_id,word,lemma,tag,word_position,inverted_word_index,inverted_lemma_index
23,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,3,Перелом случился в XX веке.,3,24,XX,XX,LAT,24,"[3, 4, 13]","[3, 4, 13]"
26,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,4,до XX века:\n - описывали грамматику и лексику...,4,27,XX,XX,LAT,27,"[3, 4, 13]","[3, 4, 13]"
43,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,4,до XX века:\n - описывали грамматику и лексику...,4,44,XIX,XIX,LAT,44,[4],[4]
129,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,10,Следующий этап:\nПсихологизм; Леонард Блумфилд...,10,130,ср,ср,LAT,130,"[10, 24, 26, 32, 46, 51, 60, 87, 94, 104, 113,...","[10, 24, 26, 32, 46, 51, 60, 87, 94, 104, 113,..."
156,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,11,яблоко (фрукт или слово); изучать реакции долж...,11,157,NaCL,NaCL,LAT,157,[11],[11]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9667,f,33,Импликатура как механизм семантических сдвигов.,f 33. Импликатура как механизм семантических с...,501,В некоторых употреблениях идея настоящего или ...,29,9668,etc,etc,LAT,366,[501],[501]
9789,f,33,Импликатура как механизм семантических сдвигов.,f 33. Импликатура как механизм семантических с...,510,"В контексте грамматики можно ещё говорить, нап...",38,9790,НСВ,НСВ,LAT,488,[510],[510]
9927,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,522,Виноградов \n(классификация восходит к Ш. Балл...,10,9928,ср,ср,LAT,115,"[10, 24, 26, 32, 46, 51, 60, 87, 94, 104, 113,...","[10, 24, 26, 32, 46, 51, 60, 87, 94, 104, 113,..."
9944,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,523,"корень зла, принимать меры, одержать победу, р...",11,9945,VS,VS,LAT,132,"[12, 39, 60, 101, 129, 368, 428, 435, 523]","[12, 39, 60, 101, 129, 368, 428, 435, 523]"


In [ ]:
nltk.pos_tag(['≈'])

[('≈', 'NN')]

In [ ]:
lat_words = LATs.word.unique()

In [ ]:
corpus_df.tag.unique()

array(['PR', 'SPRO', 'S', 'PART', 'V', 'A', 'ADV', 'ADVPRO', 'LAT',
       'CONJ', 'APRO', 'INTJ', 'NUM', 'ANUM'], dtype=object)

In [ ]:
latposes = []

for i in lat_words:
  if i.isalpha():
    latposes.append(nltk.pos_tag([i])[0])

In [ ]:
latposes

[('xx', 'NN'),
 ('xix', 'NN'),
 ('ср', 'NN'),
 ('nacl', 'NN'),
 ('дж', 'NN'),
 ('specific', 'JJ'),
 ('definite', 'NN'),
 ('indefinite', 'NN'),
 ('ground', 'NN'),
 ('vs', 'NN'),
 ('figure', 'NN'),
 ('trajector', 'NN'),
 ('landmark', 'NN'),
 ('langacker', 'NN'),
 ('go', 'VB'),
 ('gone', 'VBN'),
 ('steal', 'NN'),
 ('rob', 'NN'),
 ('f', 'NN'),
 ('or', 'CC'),
 ('the', 'DT'),
 ('act', 'NN'),
 ('of', 'IN'),
 ('cooking', 'NN'),
 ('a', 'DT'),
 ('cook', 'NN'),
 ('food', 'NN'),
 ('being', 'VBG'),
 ('cooked', 'NNS'),
 ('container', 'NN'),
 ('to', 'TO'),
 ('hold', 'NN'),
 ('while', 'IN'),
 ('it', 'PRP'),
 ('is', 'VBZ'),
 ('heating', 'NN'),
 ('instrument', 'NN'),
 ('framenet', 'NN'),
 ('frame', 'NN'),
 ('elements', 'NNS'),
 ('fe', 'NN'),
 ('lexical', 'JJ'),
 ('units', 'NNS'),
 ('lu', 'NN'),
 ('placing', 'VBG'),
 ('theme', 'NN'),
 ('goal', 'NN'),
 ('agent', 'NN'),
 ('cause', 'NN'),
 ('revenge', 'NN'),
 ('offender', 'NN'),
 ('injury', 'NN'),
 ('injured', 'JJ'),
 ('party', 'NN'),
 ('avenger', 'NN'),
 (

In [ ]:
set(latposes)

{'CC',
 'DT',
 'IN',
 'JJ',
 'NN',
 'NNS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'TO',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBZ'}